In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
import numpy as np
import requests
import re

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent


# chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
# os.environ["webdriver.chrome.driver"] = chromedriver
# driver = webdriver.Chrome(chromedriver)
# base_url = 'https://www.zillow.com/homes/chicago_rb/'

In [ ]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [4]:
def get_property_urls(link):
    # omg thank god for some dude on github for this req_header 0 CAPTCHAS i owe you my life
    
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }


    r = requests.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    table = soup.find('ul', class_='photo-cards photo-cards_wow photo-cards_short')
    li_tags = table.find_all('li')
    properties = [item.find('div', class_='list-card-info')for item in li_tags]

    property_urls = []

    for item in properties:
        if item != None:
            property_extensions = item.find('a')['href']
            property_urls.append(property_extensions)
            
    return property_urls

In [5]:
def get_property_features(link):
    try:
    
        req_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.8',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }


        r = requests.get(link, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


        # this is the code for extracting features our of the home fact list table
        all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
        features_list = [item.text.split(':') for item in all_features]
        price = soup.find('div', class_='ds-home-details-chip').find('span', class_='ds-value').text
        features_list.append(['Total Price', price])

        # this is code for extracting number of bedroom/baths from a separate div
        # then formatting it to match our features from the first table and appending
        # to feature list
        beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
        beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
        bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]


        for item in bed_baths_formatted:
            features_list.append(item.split(':'))

        print(features_list)
        return features_list
    except:
        return None


In [6]:
base_url = 'https://www.zillow.com/homes/chicago_rb/'
first_20_pages_ext = [base_url + str(i) + '_p' for i in range(2,21)]

In [7]:
all_pages_property_urls = []
for link in first_20_pages_ext:
    all_pages_property_urls.append(get_property_urls(link))

In [8]:
# so right now i have a list of list and i want to flatten it for convenience
all_urls = np.array(all_pages_property_urls).flatten()
all_urls[:10]

array(['https://www.zillow.com/homedetails/1701-N-Washtenaw-Ave-APT-1W-Chicago-IL-60647/87715857_zpid/',
       'https://www.zillow.com/homedetails/1617-W-37th-St-Chicago-IL-60609/2099090920_zpid/',
       'https://www.zillow.com/homedetails/405-N-Wabash-Ave-UNIT-1809-Chicago-IL-60611/3865057_zpid/',
       'https://www.zillow.com/homedetails/1920-W-Diversey-Pkwy-3-Chicago-IL-60614/89929073_zpid/',
       'https://www.zillow.com/homedetails/1400-W-Monroe-St-6G-Chicago-IL-60607/2077642883_zpid/',
       'https://www.zillow.com/homedetails/6923-N-Overhill-Ave-Chicago-IL-60631/2089942962_zpid/',
       'https://www.zillow.com/homedetails/618-W-Oakdale-Ave-APT-3-Chicago-IL-60657/58361256_zpid/',
       'https://www.zillow.com/homedetails/4969-N-Kolmar-Ave-Chicago-IL-60630/2085298934_zpid/',
       'https://www.zillow.com/homedetails/4341-N-Mcvicker-Ave-Chicago-IL-60634/2085257443_zpid/',
       'https://www.zillow.com/homedetails/460-W-Huron-St-Chicago-IL-60654/70427044_zpid/'],
      dtyp

In [17]:
test = all_urls[0].replace('https://www.zillow.com/homedetails/','').split('/')[0]
test

'1701-N-Washtenaw-Ave-APT-1W-Chicago-IL-60647'

In [15]:
test

'1701-N-Washtenaw-Ave-APT-1W-Chicago-IL-60647/87715857_zpid/'

In [32]:
data_list = []
bum_links = []
for link in all_urls:
    print(link)
    data_raw = get_property_features(link)
    
    if data_raw != None:
        # adding the dict down here so we dont throw an error when something
        # doesnt come up/can have conditions
        data = dict(data_raw)
        data['Address'] = link.replace('https://www.zillow.com/homedetails/','').split('/')[0]
        data['URL'] = link
        data_list.append(data)
    else:
        # creating this so we can see what links didnt work and look into why later
        bum_links.append(link)
        

https://www.zillow.com/homedetails/1701-N-Washtenaw-Ave-APT-1W-Chicago-IL-60647/87715857_zpid/
[['Type', 'Condo'], ['Year built', '2005'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$227/mo'], ['Total Price', '$500,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/1617-W-37th-St-Chicago-IL-60609/2099090920_zpid/
[['Type', 'Multiple Occupancy'], ['Year built', '1925'], ['Heating', 'Other, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Price/sqft', '$221'], ['Total Price', '$448,800'], ['Bedrooms', ' 4'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3']]
https://www.zillow.com/homedetails/405-N-Wabash-Ave-UNIT-1809-Chicago-IL-60611/3865057_zpid/
[['Type', 'Condo'], ['Year built', '1977'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$549/mo'], ['Total Price', '$250,000'], ['Bedrooms', ' 1'], ['Bathrooms', ' 

[['Type', 'Townhouse'], ['Year built', '1989'], ['Heating', 'Electric, Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['Lot', '0.57 Acres'], ['Price/sqft', '$291'], ['Total Price', '$719,900'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/2628-N-Ridgeway-Ave-Chicago-IL-60647/2077643793_zpid/
[['Type', 'Multiple Occupancy'], ['Year built', '1913'], ['Heating', 'Forced air, Gas'], ['Cooling', 'No Data'], ['Parking', 'No Data'], ['Total Price', '$690,000'], ['Bedrooms', ' 6'], ['Bathrooms', ' 0']]
https://www.zillow.com/homedetails/130-S-Canal-St-APT-502-Chicago-IL-60606/58368776_zpid/
[['Type', 'Condo'], ['Year built', '1999'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$970/mo'], ['Price/sqft', '$333'], ['Total Price', '$500,000'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/1844-N-Fremont-St-Ch

[['Type', 'Condo'], ['Year built', '1996'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', '$137/month'], ['Price/sqft', 'No Data'], ['Total Price', '$435,000'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/1651-N-Harding-Ave-Chicago-IL-60647/2077645369_zpid/
[['Type', 'Multi Family'], ['Year built', '1904'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', '2 spaces'], ['Price/sqft', 'No Data'], ['Total Price', '$274,900'], ['Bedrooms', ' 4'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3']]
https://www.zillow.com/homedetails/4646-N-Greenview-Ave-UNIT-8-Chicago-IL-60640/60263164_zpid/
[['Type', 'Townhouse'], ['Year built', '2001'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Lot', '1.46 Acres'], ['Price/sqft', '$241'], ['Total Price', '$539,900'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3']]
https://www.zillow.com/homedetails/1

[['Type', 'Condo'], ['Year built', '2016'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$211/month'], ['Price/sqft', '$213'], ['Total Price', '$489,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/550-N-Kingsbury-St-APT-419-Chicago-IL-60654/3862031_zpid/
[['Type', 'Condo'], ['Year built', '1920'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', '$609/month'], ['Price/sqft', '$421'], ['Total Price', '$590,000'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/735-W-Wellington-Ave-3-Chicago-IL-60657/3720464_zpid/
[['Type', 'Condo'], ['Year built', '1920'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', 'No Data'], ['HOA', '$470/mo'], ['Price/sqft', '$349'], ['Total Price', '$349,000'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homede

[['Type', 'Single Family'], ['Year built', '1876'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', 'No Data'], ['Lot', '1,973 sqft'], ['Price/sqft', '$451'], ['Total Price', '$725,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/3800-N-Lake-Shore-Dr-14D-Chicago-IL-60613/3714308_zpid/
[['Type', 'Condo'], ['Year built', '1927'], ['Heating', 'Radiant, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', '$1,622/mo'], ['Price/sqft', '$340'], ['Total Price', '$749,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3']]
https://www.zillow.com/homedetails/9640-S-Woodlawn-Ave-Chicago-IL-60628/4134999_zpid/
[['Type', 'Single Family'], ['Year built', '1944'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', '2 spaces'], ['Lot', '3,092 sqft'], ['Price/sqft', '$186'], ['Total Price', '$210,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.c

...

[['Type', 'Condo'], ['Year built', '1925'], ['Heating', 'Forced air, Electric'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$560/mo'], ['Price/sqft', '$257'], ['Total Price', '$449,500'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/1725-W-90th-Pl-Chicago-IL-60620/4123694_zpid/
[['Type', 'Single Family'], ['Year built', '1950'], ['Heating', 'Forced air'], ['Cooling', 'No Data'], ['Parking', '2 spaces'], ['Lot', '3,100 sqft'], ['Price/sqft', '$175'], ['Total Price', '$175,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 1'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/5932-S-Talman-Ave-Chicago-IL-60629/3933211_zpid/
[['Type', 'Single Family'], ['Year built', '1925'], ['Heating', 'No Data'], ['Cooling', 'No Data'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '3,754 sqft'], ['Price/sqft', '$134'], ['Total Price', '$215,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 2'], ['Full bat

[['Type', 'Condo'], ['Year built', '1970'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$583/month'], ['Price/sqft', '$313'], ['Total Price', '$234,900'], ['Bedrooms', ' 1'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1']]
https://www.zillow.com/homedetails/2604-S-Emerald-Ave-B-Chicago-IL-60616/3877485_zpid/
[['Type', 'Townhouse'], ['Year built', '1994'], ['Heating', 'Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', 'None'], ['Lot', '793 sqft'], ['Price/sqft', '$287'], ['Total Price', '$319,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 1'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/5731-W-Huron-St-Chicago-IL-60644/3807213_zpid/
[['Type', 'Single Family'], ['Year built', '1900'], ['Heating', 'No Data'], ['Cooling', 'No Data'], ['Parking', '4 spaces'], ['Lot', '5,183 sqft'], ['Price/sqft', '$127'], ['Total Price', '$230,000'], ['Bedrooms', ' 4'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 1'], [

[['Type', 'Single Family'], ['Year built', '1887'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', '6 spaces'], ['Lot', '0.32 Acres'], ['Price/sqft', '$391'], ['Total Price', '$1,515,000'], ['Bedrooms', ' 5'], ['Bathrooms', ' 4'], ['Full bathrooms', ' 3'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/1360-N-Sandburg-Ter-2609-Chicago-IL-60610/2077704272_zpid/
[['Type', 'Condo'], ['Year built', '1965'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', 'None'], ['HOA', '$1,608/month'], ['Price/sqft', '$222'], ['Total Price', '$400,000'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3']]
https://www.zillow.com/homedetails/9402-S-Normal-Ave-Chicago-IL-60620/4119855_zpid/
[['Type', 'Single Family'], ['Year built', '1959'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Lot', '3,123 sqft'], ['Price/sqft', '$167'], ['Total Price', '$239,900'], ['Bedrooms', ' 4'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], 

[['Type', 'Condo'], ['Year built', '1964'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$779/month'], ['Price/sqft', '$279'], ['Total Price', '$229,875'], ['Bedrooms', ' 1'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1']]
https://www.zillow.com/homedetails/4744-W-Grace-St-Chicago-IL-60641/2140999089_zpid/
[['Type', 'Multiple Occupancy'], ['Year built', '1911'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['Price/sqft', '$300'], ['Total Price', '$449,900'], ['Bedrooms', ' 4'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/4117-W-Cullerton-St-Chicago-IL-60623/2083749817_zpid/
[['Type', 'Multi Family'], ['Year built', '1905'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', '3 spaces'], ['Price/sqft', 'No Data'], ['Total Price', '$160,000'], ['Bedrooms', ' 5'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1']]
https://www.zillow.com/homedetails/41

[['Type', 'Condo'], ['Year built', 'No Data'], ['Heating', 'No Data'], ['Cooling', 'No Data'], ['Parking', '1 space'], ['Price/sqft', '$691'], ['Total Price', '$830,000'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/1742-W-Pierce-Ave-Chicago-IL-60622/3859902_zpid/
[['Type', 'Single Family'], ['Year built', '1999'], ['Heating', 'Forced air, Radiant'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '2,400 sqft'], ['Price/sqft', '$568'], ['Total Price', '$1,200,000'], ['Bedrooms', ' 4'], ['Bathrooms', ' 4']]
https://www.zillow.com/homedetails/2905-N-Halsted-St-APT-401-Chicago-IL-60657/89929047_zpid/
[['Type', 'Condo'], ['Year built', '2006'], ['Heating', 'Forced air, Gas'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$393/mo'], ['Price/sqft', '$397'], ['Total Price', '$624,900'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2']]
https://www.zillow.com/homedetails/720-W-Randol

In [42]:
data_list
# get rid of nones 
# data_list_clean = list(filter(None, data_list)) 
df = pd.DataFrame(data_list)
df.to_csv('zillow_data.csv', header=True)

In [38]:
df

,Type,Year built,Heating,Cooling,Parking,HOA,Total Price,Bedrooms,Bathrooms,Full bathrooms,Half bathrooms,Address,URL,Price/sqft,Lot,Date available,Pets,Laundry,Deposit & fees
0,Condo,2005,"Forced air, Gas",Central,1 space,$227/mo,"$500,000",3,3,2,1,1701-N-Washtenaw-Ave-APT-1W-Chicago-IL-60647,https://www.zillow.com/homedetails/1701-N-Wash...,NaN,NaN,NaN,NaN,NaN,NaN
1,Multiple Occupancy,1925,"Other, Gas",Central,2 spaces,NaN,"$448,800",4,3,3,NaN,1617-W-37th-St-Chicago-IL-60609,https://www.zillow.com/homedetails/1617-W-37th...,$221,NaN,NaN,NaN,NaN,NaN
2,Condo,1977,"Forced air, Gas",Central,1 space,$549/mo,"$250,000",1,1,1,NaN,405-N-Wabash-Ave-UNIT-1809-Chicago-IL-60611,https://www.zillow.com/homedetails/405-N-Wabas...,NaN,NaN,NaN,NaN,NaN,NaN
3,Condo,2007,Gas,Central,1 space,$243/month,"$487,500",2,2,2,NaN,1920-W-Diversey-Pkwy-3-Chicago-IL-60614,https://www.zillow.com/homedetails/1920-W-Dive...,No Data,NaN,NaN,NaN,NaN,NaN
4,Condo,No Data,"Forced air, Gas",Central,42 spaces,$547/mo,"$1,390,000",3,3,3,NaN,1400-W-Monroe-St-6G-Chicago-IL-60607,https://www.zillow.com/homedetails/1400-W-Monr...,$710,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,Multiple Occupancy,1916,Gas,No Data,6 spaces,NaN,"$1,049,000",5,3,3,NaN,3815-N-Lakewood-Ave-Chicago-IL-60613,https://www.zillow.com/homedetails/3815-N-Lake...,NaN,NaN,NaN,NaN,NaN,NaN
736,Condo,2020,"Forced air, Gas",Central,1 space,NaN,"$569,000",3,2,2,NaN,5652-N-Ashland-Ave-2-Chicago-IL-60660,https://www.zillow.com/homedetails/5652-N-Ashl...,$406,NaN,NaN,NaN,NaN,NaN
737,Condo,1889,Forced air,Central,1 space,$325/month,"$635,000",3,2,2,NaN,626-W-Belden-Ave-3-Chicago-IL-60614,https://www.zillow.com/homedetails/626-W-Belde...,"$635,000",NaN,NaN,NaN,NaN,NaN
738,Single Family,1922,Forced air,Central,2 spaces,NaN,"$189,900",5,2,2,NaN,Chicago-IL-60620,https://www.zillow.com/homedetails/Chicago-IL-...,No Data,"3,092 sqft",NaN,NaN,NaN,NaN


### Get info from facts and features table on individual property

In [51]:
a = pd.read_csv('zillow_data.csv').drop(columns=['Unnamed: 0'])
a

,Type,Year built,Heating,Cooling,Parking,HOA,Total Price,Bedrooms,Bathrooms,Full bathrooms,Half bathrooms,Address,URL,Price/sqft,Lot,Date available,Pets,Laundry,Deposit & fees
0,Condo,2005,"Forced air, Gas",Central,1 space,$227/mo,"$500,000",3,3.0,2.0,1.0,1701-N-Washtenaw-Ave-APT-1W-Chicago-IL-60647,https://www.zillow.com/homedetails/1701-N-Wash...,NaN,NaN,NaN,NaN,NaN,NaN
1,Multiple Occupancy,1925,"Other, Gas",Central,2 spaces,NaN,"$448,800",4,3.0,3.0,NaN,1617-W-37th-St-Chicago-IL-60609,https://www.zillow.com/homedetails/1617-W-37th...,$221,NaN,NaN,NaN,NaN,NaN
2,Condo,1977,"Forced air, Gas",Central,1 space,$549/mo,"$250,000",1,1.0,1.0,NaN,405-N-Wabash-Ave-UNIT-1809-Chicago-IL-60611,https://www.zillow.com/homedetails/405-N-Wabas...,NaN,NaN,NaN,NaN,NaN,NaN
3,Condo,2007,Gas,Central,1 space,$243/month,"$487,500",2,2.0,2.0,NaN,1920-W-Diversey-Pkwy-3-Chicago-IL-60614,https://www.zillow.com/homedetails/1920-W-Dive...,No Data,NaN,NaN,NaN,NaN,NaN
4,Condo,No Data,"Forced air, Gas",Central,42 spaces,$547/mo,"$1,390,000",3,3.0,3.0,NaN,1400-W-Monroe-St-6G-Chicago-IL-60607,https://www.zillow.com/homedetails/1400-W-Monr...,$710,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,Multiple Occupancy,1916,Gas,No Data,6 spaces,NaN,"$1,049,000",5,3.0,3.0,NaN,3815-N-Lakewood-Ave-Chicago-IL-60613,https://www.zillow.com/homedetails/3815-N-Lake...,NaN,NaN,NaN,NaN,NaN,NaN
736,Condo,2020,"Forced air, Gas",Central,1 space,NaN,"$569,000",3,2.0,2.0,NaN,5652-N-Ashland-Ave-2-Chicago-IL-60660,https://www.zillow.com/homedetails/5652-N-Ashl...,$406,NaN,NaN,NaN,NaN,NaN
737,Condo,1889,Forced air,Central,1 space,$325/month,"$635,000",3,2.0,2.0,NaN,626-W-Belden-Ave-3-Chicago-IL-60614,https://www.zillow.com/homedetails/626-W-Belde...,"$635,000",NaN,NaN,NaN,NaN,NaN
738,Single Family,1922,Forced air,Central,2 spaces,NaN,"$189,900",5,2.0,2.0,NaN,Chicago-IL-60620,https://www.zillow.com/homedetails/Chicago-IL-...,No Data,"3,092 sqft",NaN,NaN,NaN,NaN


In [40]:
len(bum_links)

20

In [ ]:
# def get_property_features(link):
#     try:
r = requests.get('https://www.zillow.com/homedetails/460-W-Huron-St-Chicago-IL-60654/70427044_zpid/', headers=req_headers)
soup = BeautifulSoup(r.content, 'html.parser')
facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


# this is the code for extracting features our of the home fact list table
all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
features_list = [item.text.split(':') for item in all_features]
price = soup.find('div', class_='ds-home-details-chip').find('span', class_='ds-value').text
features_list.append(['Total Price', price])

# this is code for extracting number of bedroom/baths from a separate div
# then formatting it to match our features from the first table and appending
# to feature list

beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]

for item in bed_baths_formatted:
    features_list.append(item.split(':'))

    
    

In [ ]:
features_list

In [ ]:
# soup.find(text='Bedrooms: ').next_element.next_element
# soup.find(text='Bathrooms: ').next_element.next_element
# soup.find(text='Bathrooms: ').next_element.next_element
re.split('(\d+)','Bedrooms: 4Bathrooms: 3Full bathrooms: 3')
# bed_baths_div = soup.find('div', class_='sc-pdKru hHHZLq')
# # for item in a:
# # #     print(item.text)
# #     print(item)
bed_baths = [item.text + ' ' for item in bed_baths_div]
bed_baths = re.split('(\d+)', a[1])[:-1]
bed_baths_formatted = [i+j for i,j in zip(test[::2], test[1::2])]
bed_baths_formatted
# for item in bed_baths_div: 
#     print(re.split('(\d+)', item.text))
# # a.find_all('ul')

In [ ]:
re.sub('(\d+)',',', 'Bedrooms: 4Bathrooms: 3Full bathrooms: 3')

In [ ]:
bed_baths_div

In [ ]:
pd.DataFrame(features_list[:,1])

In [ ]:
summary_info

In [ ]:
ds-home-fact-list-item

In [ ]:
facts_and_features_table.find('ul', class_='ds-home-fact-list')

In [ ]:
pd.read_csv('zillow_data')

In [ ]:
# req_headers = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#     'accept-encoding': 'gzip, deflate, br',
#     'accept-language': 'en-US,en;q=0.8',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
# }

In [ ]:
# req_headers = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#     'accept-encoding': 'gzip, deflate, br',
#     'accept-language': 'en-US,en;q=0.8',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
# }

# with requests.Session() as s:
#     city = 'chicago/'

#     url = 'https://www.zillow.com/homes/for_sale/'+city
#     url2 = 'https://www.zillow.com/homes/for_sale/'+city+'/2_p/'
#     url3 = 'https://www.zillow.com/homes/for_sale/'+city+'/3_p/'
#     url4 = 'https://www.zillow.com/homes/for_sale/'+city+'/4_p/'
#     url5 = 'https://www.zillow.com/homes/for_sale/'+city+'/5_p/'
#     url6 = 'https://www.zillow.com/homes/for_sale/'+city+'/6_p/'
#     url7 = 'https://www.zillow.com/homes/for_sale/'+city+'/7_p/'
#     url8 = 'https://www.zillow.com/homes/for_sale/'+city+'/8_p/'
#     url9 = 'https://www.zillow.com/homes/for_sale/'+city+'/9_p/'
#     url10 = 'https://www.zillow.com/homes/for_sale/'+city+'/10_p/'

#     r = s.get(url, headers=req_headers)
#     r2 = s.get(url2, headers=req_headers)
#     r3 = s.get(url3, headers=req_headers)
#     r4 = s.get(url4, headers=req_headers)
#     r5 = s.get(url5, headers=req_headers)
#     r6 = s.get(url6, headers=req_headers)
#     r7 = s.get(url7, headers=req_headers)
#     r8 = s.get(url8, headers=req_headers)
#     r9 = s.get(url9, headers=req_headers)
#     r10 = s.get(url10, headers=req_headers)

#     url_links = [url, url2, url3, url4, url5, url6, url7, url8, url9, url10]

# #add contents of urls to soup variable from each url
# soup = BeautifulSoup(r.content, 'html.parser')
# soup1 = BeautifulSoup(r2.content, 'html.parser')
# soup2 = BeautifulSoup(r3.content, 'html.parser')
# soup3 = BeautifulSoup(r4.content, 'html.parser')
# soup4 = BeautifulSoup(r5.content, 'html.parser')
# soup5 = BeautifulSoup(r6.content, 'html.parser')
# soup6 = BeautifulSoup(r7.content, 'html.parser')
# soup7 = BeautifulSoup(r8.content, 'html.parser')
# soup8 = BeautifulSoup(r9.content, 'html.parser')
# soup9 = BeautifulSoup(r10.content, 'html.parser')
